In [1]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf

true_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//true_array.npy')
false_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//false_array.npy')



In [2]:
true_stand_data=true_stand_data[:,:,:,0]
false_stand_data=false_stand_data[:,:,:,0]

In [3]:
true_stand_data=true_stand_data.swapaxes(1, 2)
false_stand_data=false_stand_data.swapaxes(1, 2)

In [4]:
sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }

In [5]:
true_stand_data.shape

(25230, 7, 110)

In [6]:
true_sample_channel_1 = []
false_sample_channel_1 = []


for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))

In [7]:
combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)


combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

In [13]:
# Model 1
in_1 = Input(shape=(7,110))
x_120_1 = Dense(units=120,input_shape=(7,110),activation='linear')(in_1)
x_120_1 = Dense(units=40,activation='linear')(x_120_1)
x_120_1 = Dense(units=10)(x_120_1)
x_120_1 = Flatten()(x_120_1)
x_120_1 = Dense(2, activation='tanh')(x_120_1)

in_2 = Input(shape=(7,110))
x_60_1 = Dense(units=60,input_shape=(7,110),activation='linear')(in_2)
x_60_1 = Dense(units=10)(x_60_1)
x_60_1 = Flatten()(x_60_1)
x_60_1 = Dense(2, activation='tanh')(x_60_1)

in_3 = Input(shape=(7,110))
x_30_1 = Dense(units=30,input_shape=(7,110),activation='linear')(in_3)
x_30_1 = Dense(units=10)(x_30_1)
x_30_1 = Flatten()(x_30_1)
x_30_1 = Dense(2, activation='tanh')(x_30_1)


in_4 = Input(shape=(7,110))
x_10_1 = Dense(units=10,input_shape=(7,110),activation='linear')(in_4)
x_10_1 = Flatten()(x_10_1)
x_10_1 = Dense(units=2, activation='tanh')(x_10_1)

x_middle_concat = Concatenate(axis=-1)([x_120_1,x_60_1,x_30_1,x_10_1])

# x_middle_concat = Dense(10, activation='tanh')(x_middle_concat)
# x_middle_concat_Flatten = Flatten()(x_middle_concat)
model_final_dense_out=Dense(2, activation='softmax')(x_middle_concat)

model = Model(inputs=[in_1,in_2,in_3,in_4], outputs=model_final_dense_out)
print(model.summary())

import os
if 'GRU_predict_v9.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v9.h5")

callbacks_list = [    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=30
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v9.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.8,
        patience=5, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=[combin_train_data1,combin_train_data1,combin_train_data1,combin_train_data1],
          y=combin_target_data,
          batch_size=36,
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list      
         )



Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 7, 110)]     0                                            
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 7, 120)       13320       input_12[0][0]                   
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 7, 110)]     0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 7, 110)]     0                                            
______________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
x_middle_concat_out_Flatten.shape

In [ ]:
x_100_1 = GRU(units=100)(x_480_1)
x_10_1 =Dense(10, activation='tanh')(x_100_1)

model_final_dense_out = Dense(2, activation='softmax')(x_10_1)
